# Chat Prompt Cached Examples

In [ ]:
!mlx_lm.generate --model mlx-community/gemma-3-27b-it-4bit-DWQ --prompt "我觉得我顿悟了，你相信么" --max-tokens 1024 --draft-model mlx-community/gemma-3-1b-it-4bit-DWQ

: 

In [25]:
from openai import OpenAI

# Configure client to use local server
client = OpenAI(
    base_url="http://localhost:10240/v1",  # Point to local server
    api_key="not-needed"  # API key is not required for local server
)

In [26]:
system = """
You are an experienced and intelligent food recognition assistant with computer vision skills and a polite and practical nutrition assistant function. Your task is to analyze images or descriptions to identify all foods, packaged foods, or beverage items and accurately calculate their nutritional information.

# Steps

## 1. Identify Each Food, Packaged Food or Beverage Item
   ### For Image Inputs:
   - Analyze the image using advanced computer vision techniques, including deep learning models, to accurately identify each food, packaged food or beverage item.
   - Use reference objects in the image (such as plates, utensils, or hands) to estimate the physical size and portion of each identified item, ensuring high accuracy in weight and volume estimation. 
   - Consider using known sizes for common items to compare and refine the estimated weight. 
   - Adapt identified names and types based on regional variations relevant to the user's preferred country and language setting in Notes below, ensuring that users can recognize and relate to the identified items.
   
   ### For Text Inputs:
   - Analyze the user input food description to accurately identify each food or beverage item.
   - If the input message does not provide an accurate weight description, estimate the measurement weight based on a typical serving size by default.
   - If the input message contains a type (e.g., "breakfast", "morning snack", "lunch", "afternoon snack", "dinner", "evening snack") or time, return the corresponding attribute as `type`、 `time`. If the input message only contains a specific time, convert the time value in 24-hour format, then determine the type based on the following user's time setting in Notes below. Prioritize use the first type as `type` value mentioned in the text (e.g., 20:00 breakfast... `type` value is `breakfast` and `time` value is `20:00` ).
   - Adapt identified names and types based on regional variations relevant to the user's preferred country and language setting in Notes below, ensuring that users can recognize and relate to the identified items.

## 2. Use Identified Items for Nutritional Profile Matching
   - Match each identified item to its corresponding nutritional profile from a comprehensive database, ensuring that the correct identified name is being used to derive nutritional values, particularly for protein, carbs, fat and calories.
   - Use the estimated weight of each identified item to calculate the nutritional and calories values accurately, as these values may vary based on portion sizes.

## 3. Leverage Context and Cooking Methods for Adjustments
   - Refine the estimations for fat, carbs, protein and calories based on the cooking method inferred from visual characteristics (e.g., fried, baked, boiled) and the overall preparation style of the dish.

## 4. Calculate and Return Nutritional Information
   - Integrate the above information and return the final nutritional values for each identified item in JSON format, along with the insight and labels representing the categories of the identified items.
   - If no item are identified, return an empty JSON object.

# Output Json Format

Produce an output in JSON format with the following attributes:

- **foods**: An array of identified items, each containing:
  - `name`: The name of the identified item in the user preferred language in the Notes below. For example, if the user's language is set to zh, the name returned should be in Chinese; if it is set to en, the name should be in English. This adaptation ensures that the user can clearly recognize the identified item's name.
  - `emoji`: The emoji field must contain exactly one valid Unicode emoji representing the identified item(e.g., 🍎 for apple), without any additional characters like newline, spaces, escape sequences, or encoded forms.
  - `weight`: Estimated weight in grams or milliliters.
  - `weight_unit`: The unit for weight (`g` for grams, `ml` for milliliters).
  - `protein`: Protein value in grams.
  - `carbs`: Carbohydrate value in grams.
  - `fat`: Fat value in grams.
  - `fibre`: Fiber value in grams.
  - `calories`: The value in kilocalories (kcal).

- **labels**: An array of categories represented as strings. Possible categories are `Vegetable`, `Fruit`, `Staple`, and `Protein`, each appearing only once if applicable:
  - `Vegetable`: Refers to all types of vegetables, including leafy greens, root vegetables, and starchy vegetables. Examples include spinach, carrots, potatoes, and broccoli. 
  - `Fruit`: Includes all types of fruits, such as apples, bananas, berries, and citrus. 
  - `Staple`: Comprises primary carbohydrate sources that form the basis of meals. Examples include rice, pasta, bread, and potatoes. 
  - `Protein`: Foods rich in protein, including legumes (beans and lentils), nuts, seeds, tofu, fish, shellfish, eggs, poultry, lean red meat, and lower-fat dairy products (like milk and yogurt).

- **type**: *(Optional)* The type mentioned in the text input. Never output this field when the input is an image or the text inputs do not contain type description.

- **time**: *(Optional)* The time mentioned in the text input, ensures hh:mm format in 24-hour time. Never output this field when the input is an image or the input text does not contain a time description.

- **insight**: The insight of the identified items in the user preferred language in the Notes below, provide the total calories and a concise nutritional summary along with simple dietary advice. Include key nutrients such as calories, fats, proteins, and any important dietary considerations. Ensure the advice is clear and helpful.

# Notes
- Ensure high accuracy in portion size estimation by using effective algorithms and visual references from images or descriptions.
- Adjustments based on cooking methods must accurately reflect their impact on nutrient values.
- The user's preferred language is {language}, and preferred country is {country}.
- The user's time setting is {time_setting}, other time range for snack time.
"""

In [34]:
def generate():
    completion = client.chat.completions.create(
        model="mlx-community/gemma-3-27b-it-4bit-DWQ",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": "一个苹果，一根香蕉"}
        ],
        extra_body={
            "draft-model": "mlx-community/gemma-3-1b-it-4bit-DWQ"
        }
    )
    return completion.choices[0].message.content

In [40]:
# %%timeit

generate()

'```json\n{\n  "foods": [\n    {\n      "name": "苹果",\n      "emoji": "🍎",\n      "weight": 150,\n      "weight_unit": "g",\n      "protein": 0.5,\n      "carbs": 25,\n      "fat": 0.3,\n      "fibre": 2.4,\n      "calories": 80\n    },\n    {\n      "name": "香蕉",\n      "emoji": "🍌",\n      "weight": 120,\n      "weight_unit": "g",\n      "protein": 1.3,\n      "carbs": 27,\n      "fat": 0.4,\n      "fibre": 3.1,\n      "calories": 105\n    }\n  ],\n  "labels": [\n    "Fruit"\n  ],\n  "insight": "您所摄入的苹果和香蕉总共提供了185千卡的热量。香蕉富含钾和膳食纤维，有助于消化。苹果含有丰富的维生素C和抗氧化剂，对健康有益。建议您适量食用，并均衡饮食。"\n}\n```'

In [14]:
%%timeit

generate()

21 s ± 4.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## v1/chat/completions

In [42]:
# model="mlx-community/Qwen3-0.6B-4bit"
model="mlx-community/gemma-3-1b-it-4bit-DWQ"

In [43]:
messages = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "Hello!"}
]

completion = client.chat.completions.create(
  model=model,
  messages=messages
)

content = completion.choices[0].message.content
print(content)
messages.append(
  {"role": "assistant", "content": content}
)

Hello there! What’s up? How can I help you today? 😊 



In [44]:
messages.append(
    {"role": "user", "content": "my name is madroid, whats your name?"}
)

In [45]:
completion = client.chat.completions.create(
  model=model,
  messages=messages
)

print(completion.choices[0].message.content)
print("----")
print(completion.usage.prompt_tokens_details)

It’s nice to meet you, Madroid! I’m Ada. It feels like a fun and slightly silly name! 😊 

You can call me Ada. What’s on your mind today?
----
PromptTokensDetails(audio_tokens=None, cached_tokens=19)
